# Working with Dates in Pandas


In [2]:
import pandas as pd
from random import randint
from datetime import timedelta, date

## Date Ranges 
- `pandas.date_range(start, end)` is the simplest use which produces a `DatetimeIndex` which can be converted to a Series object
    - see [the official documentation](https://pandas.pydata.org/docs/reference/api/pandas.date_range.html)

### Date Range Examples

#### Date Range from Date Strings

In [3]:
# Daily Range
dr = pd.date_range('2000-01-01', '2000-01-03')
dr

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03'], dtype='datetime64[ns]', freq='D')

#### From Date Objects

In [4]:
# Daily Range from Date types
dr = pd.date_range(date(2000, 1, 1), date(2000, 1, 3))
dr

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03'], dtype='datetime64[ns]', freq='D')

#### With a Monthly Frequency
See [frequency string](https://pandas.pydata.org/pandas-docs/dev/user_guide/timeseries.html#timeseries-offset-aliases) documentation for more information.

In [5]:
# Month end Frequency
dr = pd.date_range(start='2000-01-01', end='2000-04-01', freq='M')
dr

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31'], dtype='datetime64[ns]', freq='M')

In [6]:
# Month end Frequency with a number of periods
dr = pd.date_range(start='2000-01-01', periods=3, freq='M')
dr

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31'], dtype='datetime64[ns]', freq='M')

In [7]:
# Month begin Frequency with a number of periods
dr = pd.date_range(start='2000-01-01', periods=3, freq='MS')
dr

DatetimeIndex(['2000-01-01', '2000-02-01', '2000-03-01'], dtype='datetime64[ns]', freq='MS')

#### With a Weekly Frequency
- Always note which day weeks start on.

In [8]:
# Weekly 
dr = pd.date_range(start='2000-01-01', periods=3, freq='W')
dr

DatetimeIndex(['2000-01-02', '2000-01-09', '2000-01-16'], dtype='datetime64[ns]', freq='W-SUN')

In [9]:
# DateTimeIndex objects have a day_name() method (locale specific)
dr.day_name()

Index(['Sunday', 'Sunday', 'Sunday'], dtype='object')

In [10]:
# Weekly with a Monday start date
dr = pd.date_range(start='2000-01-01', periods=3, freq='W-MON')
dr

DatetimeIndex(['2000-01-03', '2000-01-10', '2000-01-17'], dtype='datetime64[ns]', freq='W-MON')

## Dates in Dataframe Columns
Here, we create a simple dataframe for the purpose of demonstrating date methods on dataframe columns.

The official documentation extensive examples for:
- [datetime-like attributes](https://pandas.pydata.org/docs/reference/series.html#datetime-properties)
- [datetime-like methods](https://pandas.pydata.org/docs/reference/series.html#datetime-methods)

In [13]:
# DatetimeIndex
start_date = pd.date_range('2000-01-01', '2000-01-05')

# Some random timedeltas (in days)
days = pd.Series([timedelta(days=randint(0, 4)) for _ in range(5)])

# Add the timedeltas to the index.  This produces a pd.Series
end_date = start_date + days

# Create a pd.DataFrame adding column names and a labeled 'event' column
df = pd.DataFrame(
    {
        'event': list('ABCDE'),
        'start_date': start_date,
        'end_date': end_date,
    }
)
df

,event,start_date,end_date
0,A,2000-01-01,2000-01-03
1,B,2000-01-02,2000-01-04
2,C,2000-01-03,2000-01-03
3,D,2000-01-04,2000-01-08
4,E,2000-01-05,2000-01-06


### Add Day Name Columns

In [14]:
df2 = (
    df
    .assign(start_day=df.start_date.dt.day_name())
    .assign(end_day=df.end_date.dt.day_name())
)
df2

,event,start_date,end_date,start_day,end_day
0,A,2000-01-01,2000-01-03,Saturday,Monday
1,B,2000-01-02,2000-01-04,Sunday,Tuesday
2,C,2000-01-03,2000-01-03,Monday,Monday
3,D,2000-01-04,2000-01-08,Tuesday,Saturday
4,E,2000-01-05,2000-01-06,Wednesday,Thursday


In [15]:
# re-order columns
df2 = df2[['event', 'start_day', 'start_date',
           'end_day', 'end_date',          
]]
df2

,event,start_day,start_date,end_day,end_date
0,A,Saturday,2000-01-01,Monday,2000-01-03
1,B,Sunday,2000-01-02,Tuesday,2000-01-04
2,C,Monday,2000-01-03,Monday,2000-01-03
3,D,Tuesday,2000-01-04,Saturday,2000-01-08
4,E,Wednesday,2000-01-05,Thursday,2000-01-06


### Add an event length column
Determine the duration of an event in days and add it to the dataframe.

In [16]:
(
    df2
    .assign(event_duration=(df2.end_date - df2.start_date).dt.days)
)
df2.end_date

0   2000-01-03
1   2000-01-04
2   2000-01-03
3   2000-01-08
4   2000-01-06
Name: end_date, dtype: datetime64[ns]

#### How it works
- `df2.end_date` is a `pd.Series` with `dtype` `datetime`. 
- `df2.end_date - df2.start_date` is a `pd.Series` with `dtype` `timedelta`.
- `timedelta` [attributes](https://pandas.pydata.org/docs/reference/series.html#timedelta-properties) provide access to the time difference in days. 

We can see the build up in the next few cells.

In [17]:
df2.end_date - df2.start_date

0   2 days
1   2 days
2   0 days
3   4 days
4   1 days
dtype: timedelta64[ns]

In [18]:
event_timedelta = df2.end_date - df2.start_date
event_timedelta

0   2 days
1   2 days
2   0 days
3   4 days
4   1 days
dtype: timedelta64[ns]

In [19]:
# use dt methods on Series with timedelta types.
event_timedelta.dt.days

0    2
1    2
2    0
3    4
4    1
dtype: int64

### Combine all operations using Chained Methods
On the original dataframe...

In [20]:
columns = ['event', 'start_day', 'start_date',
           'end_day', 'end_date', 'event_duration']

(
    df
    .assign(start_day=df.start_date.dt.day_name())
    .assign(end_day=df.end_date.dt.day_name())
    .assign(event_duration=(df.end_date - df.start_date).dt.days)
    .reindex(columns=columns) # Change the order of the columns
)

,event,start_day,start_date,end_day,end_date,event_duration
0,A,Saturday,2000-01-01,Monday,2000-01-03,2
1,B,Sunday,2000-01-02,Tuesday,2000-01-04,2
2,C,Monday,2000-01-03,Monday,2000-01-03,0
3,D,Tuesday,2000-01-04,Saturday,2000-01-08,4
4,E,Wednesday,2000-01-05,Thursday,2000-01-06,1


## Reading and Writing Data to CSV Files
This section focuses on a few tasks:
- writing a dataframe to CSV file
- reading a CSV file with dates
- writing a subset of columns to a CSV file


### Writing a DataFrame to CSV
Use the `to_csv()` method of a DataFrame, but be sure to pass `index=False` to prevent the integer index from being written to output. 

In [26]:
# Here's the DF we will work with
columns = ['event', 'start_day', 'start_date',
           'end_day', 'end_date', 'event_duration']

df_out = (
    df
    .assign(start_day=df.start_date.dt.day_name())
    .assign(end_day=df.end_date.dt.day_name())
    .assign(event_duration=(df.end_date - df.start_date).dt.days)
    .reindex(columns=columns) # Change the order of the columns
)
df_out

,event,start_day,start_date,end_day,end_date,event_duration
0,A,Saturday,2000-01-01,Monday,2000-01-03,2
1,B,Sunday,2000-01-02,Tuesday,2000-01-04,2
2,C,Monday,2000-01-03,Monday,2000-01-03,0
3,D,Tuesday,2000-01-04,Saturday,2000-01-08,4
4,E,Wednesday,2000-01-05,Thursday,2000-01-06,1


In [27]:
# Write to CSV output.csv
df_out.to_csv('output.csv', index=False)

### Inspect the result

In [28]:
%cat output.csv

event,start_day,start_date,end_day,end_date,event_duration
A,Saturday,2000-01-01,Monday,2000-01-03,2
B,Sunday,2000-01-02,Tuesday,2000-01-04,2
C,Monday,2000-01-03,Monday,2000-01-03,0
D,Tuesday,2000-01-04,Saturday,2000-01-08,4
E,Wednesday,2000-01-05,Thursday,2000-01-06,1


### Reading a CSV with Dates
Be sure to pass the `parse_date` kwarg with a list of date columns.

Here, the `output.csv` file from above is read into a DataFrame. 

In [29]:
# Read a CSV file with dates
df_in = pd.read_csv('output.csv', parse_dates=['start_date', 'end_date'])
df_in

,event,start_day,start_date,end_day,end_date,event_duration
0,A,Saturday,2000-01-01,Monday,2000-01-03,2
1,B,Sunday,2000-01-02,Tuesday,2000-01-04,2
2,C,Monday,2000-01-03,Monday,2000-01-03,0
3,D,Tuesday,2000-01-04,Saturday,2000-01-08,4
4,E,Wednesday,2000-01-05,Thursday,2000-01-06,1


### Writing a subset of columns to CSV
What if we only wanted to write part of the dataframe out to CSV.  This can be done by indexing the dataframe columns with a `list` of column names. 

Here, we only want to write out the `event` and `event_duration` columns. 

In [30]:
# subset the DF and write to CSV
columns_to_write = ['event', 'event_duration']

df_out[columns_to_write].to_csv('output-subset.csv', index=False)

In [31]:
%cat output-subset.csv

event,event_duration
A,2
B,2
C,0
D,4
E,1
